In [1]:
import wandb
import json
from pathlib import Path
import pandas as pd
import random
import numpy as np

import spacy
from spacy.tokens import Span, Doc
from spacy import displacy
from spacy.lang.en import English

from IPython.core.display import display, HTML

from utils import replace_ents_with_labels, \
                  replace_labels_with_ents, \
                  mark_ent_label_tokens, \
                  tokenize_df_with_spacy, \
                  visualize_ents, \
                  UNIQUE_LABELS 

/tmp/ipykernel_3547505/1322636561.py:13: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Load generated texts and entities

In [26]:
texts_load_path = Path("../data/essay/rewrited_train.json")


with open(texts_load_path, "r") as file:
    data = json.load(file)
    generated_texts_df = pd.DataFrame().from_records(data)
generated_texts_df.head(2)

,prompt,sampling_params,generated_text,generated_text_with_ent_labels,vizualization,tokens,trailing_whitespace,labels,STREET_ADDRESS,PHONE_NUM,NAME_STUDENT,USERNAME,URL_PERSONAL,ID_NUM,EMAIL
0,<s>[INST] Mind Mapping Tool\n\nChallenge:\n\nE...,"{'max_tokens': 4000, 'n': 1, 'temperature': 1,...","In the bustling city of London, at the famous...","In the bustling city of London, at the famous...","{'_type': 'html-file', 'path': 'media/html/361...","[ , In, the, bustling, city, of, London, ,, at...","[False, True, True, True, True, True, False, T...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]",True,None,None,None,None,None,None
1,<s>[INST] Project Title: Learning Modalities i...,"{'max_tokens': 4000, 'n': 1, 'temperature': 1,...",Project Title: Learning Modalities in the Pan...,Project Title: Learning Modalities in the Pan...,"{'_type': 'html-file', 'path': 'media/html/f9e...","[ , Project, Title, :, Learning, Modalities, i...","[False, True, False, True, True, True, True, T...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",None,True,None,None,None,None,None


In [27]:
load_path = "../data/faker_pii.csv"

generated_ents_df = pd.read_csv(load_path).drop(columns="COUNTRY")
generated_ents_combs_df = generated_ents_df.applymap(lambda x: [x]).agg(lambda row: row.to_dict(), axis=1).reset_index(drop=True)
generated_ents_combs_df.head(2)

/tmp/ipykernel_3547505/3603450406.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  generated_ents_combs_df = generated_ents_df.applymap(lambda x: [x]).agg(lambda row: row.to_dict(), axis=1).reset_index(drop=True)


0    {'ID_NUM': ['663507407PfsCnkrMYSdqsTYt'], 'NAM...
1    {'ID_NUM': ['8534955202OgSEmlhbAmTrjfr'], 'NAM...
dtype: object

# Replace entity-holders with new entities

Create true_ents_dict if absent

In [28]:
LABEL2ENT

{'NAME_STUDENT': 'James Brown',
 'EMAIL': 'example@email.com',
 'URL_PERSONAL': 'https://example.com',
 'USERNAME': 'john42',
 'STREET_ADDRESS': '221B, Baker Street, London',
 'PHONE_NUM': '+1 212 555 0188',
 'ID_NUM': '123456789'}

In [29]:
generated_texts_df

,prompt,sampling_params,generated_text,generated_text_with_ent_labels,vizualization,tokens,trailing_whitespace,labels,STREET_ADDRESS,PHONE_NUM,NAME_STUDENT,USERNAME,URL_PERSONAL,ID_NUM,EMAIL
0,<s>[INST] Mind Mapping Tool\n\nChallenge:\n\nE...,"{'max_tokens': 4000, 'n': 1, 'temperature': 1,...","In the bustling city of London, at the famous...","In the bustling city of London, at the famous...","{'_type': 'html-file', 'path': 'media/html/361...","[ , In, the, bustling, city, of, London, ,, at...","[False, True, True, True, True, True, False, T...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]",True,None,None,None,None,None,None
1,<s>[INST] Project Title: Learning Modalities i...,"{'max_tokens': 4000, 'n': 1, 'temperature': 1,...",Project Title: Learning Modalities in the Pan...,Project Title: Learning Modalities in the Pan...,"{'_type': 'html-file', 'path': 'media/html/f9e...","[ , Project, Title, :, Learning, Modalities, i...","[False, True, False, True, True, True, True, T...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",None,True,None,None,None,None,None
2,<s>[INST] I’m managing team of analysts suppor...,"{'max_tokens': 4000, 'n': 1, 'temperature': 1,...",As a manager of a team of analysts who suppor...,As a manager of a team of analysts who suppor...,"{'_type': 'html-file', 'path': 'media/html/d70...","[ , As, a, manager, of, a, team, of, analysts,...","[False, True, True, True, True, True, True, Tr...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",True,None,False,None,None,None,None
3,<s>[INST] Assignment: Mindmapping\n\nChallenge...,"{'max_tokens': 4000, 'n': 1, 'temperature': 1,...",Assignment: Mindmapping\n\nChallenge\n\nThe c...,Assignment: Mindmapping\n\nChallenge\n\nThe c...,"{'_type': 'html-file', 'path': 'media/html/2c0...","[ , Assignment, :, Mindmapping, \n\n, Challeng...","[False, False, True, False, False, False, Fals...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",None,None,False,False,None,None,None
4,<s>[INST] Challenges\n\nI am leading an educat...,"{'max_tokens': 4000, 'n': 1, 'temperature': 1,...",As the leader of the educational domain entit...,As the leader of the educational domain entit...,"{'_type': 'html-file', 'path': 'media/html/d99...","[ , As, the, leader, of, the, educational, dom...","[False, True, True, True, True, True, True, Tr...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST...",None,None,False,False,False,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5857,<s>[INST] Effective workflow management within...,"{'max_tokens': 4000, 'n': 1, 'temperature': 1,...",Challenge:\n\nAs a member of the online marke...,Challenge:\n\nAs a member of the online marke...,"{'_type': 'html-file', 'path': 'media/html/c64...","[ , Challenge, :, \n\n, As, a, member, of, the...","[False, False, False, False, True, True, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",None,None,False,None,None,False,None
5858,<s>[INST] Visualization Tool\n\nChallenge\n\nM...,"{'max_tokens': 4000, 'n': 1, 'temperature': 1,...","As an art director, my latest project involve...","As an art director, my latest project involve...","{'_type': 'html-file', 'path': 'media/html/80e...","[ , As, an, art, director, ,, my, latest, proj...","[False, True, True, True, False, True, True, T...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",None,None,True,False,False,None,None
5859,<s>[INST] Challenges\n\nWe are a business whic...,"{'max_tokens': 4000, 'n': 1, 'temperature': 1,...","As a Training Business Manager at my company,...","As a Training Business Manager at my company,...","{'_type': 'html-file', 'path': 'media/html/238...","[ , As, a, Training, Business, Manager, at, my...","[False, True, True, True, True, True, True, Tr...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",True,None,False,None,None,None,None
5860,<s>[INST] 1. Challenge: Describe your challeng...,"{'max_tokens': 4000, 'n': 1, 'temperature': 1,...","1. Challenge: Describe your challenge, includ...","1

In [30]:
for i, row in generated_texts_df.iterrows():
    break

In [33]:
PII_ENTS = [
  ("name", "NAME_STUDENT", "James Brown"),
  ("email", "EMAIL", "example@email.com"),
  ("personal_url", "URL_PERSONAL", "https://example.com"),
  ("username", "USERNAME", "john42"),
  ("address", "STREET_ADDRESS", "221B, Baker Street, London"),
  ("phone_num", "PHONE_NUM", "+1 212 555 0188"),
  ("userid", "ID_NUM", "123456789"),
]

LABEL2ENT = {l: e for _, l, e in PII_ENTS}

def add_label_dict(row):
    row["true_ents_dict"] = {label: [LABEL2ENT[label]] for label in UNIQUE_LABELS if row[label] is not None}
    return row

generated_texts_df = generated_texts_df.agg(add_label_dict, axis=1)

In [34]:
generated_texts_df

,prompt,sampling_params,generated_text,generated_text_with_ent_labels,vizualization,tokens,trailing_whitespace,labels,STREET_ADDRESS,PHONE_NUM,NAME_STUDENT,USERNAME,URL_PERSONAL,ID_NUM,EMAIL,true_ents_dict
0,<s>[INST] Mind Mapping Tool\n\nChallenge:\n\nE...,"{'max_tokens': 4000, 'n': 1, 'temperature': 1,...","In the bustling city of London, at the famous...","In the bustling city of London, at the famous...","{'_type': 'html-file', 'path': 'media/html/361...","[ , In, the, bustling, city, of, London, ,, at...","[False, True, True, True, True, True, False, T...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]",True,None,None,None,None,None,None,"{'STREET_ADDRESS': ['221B, Baker Street, Londo..."
1,<s>[INST] Project Title: Learning Modalities i...,"{'max_tokens': 4000, 'n': 1, 'temperature': 1,...",Project Title: Learning Modalities in the Pan...,Project Title: Learning Modalities in the Pan...,"{'_type': 'html-file', 'path': 'media/html/f9e...","[ , Project, Title, :, Learning, Modalities, i...","[False, True, False, True, True, True, True, T...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",None,True,None,None,None,None,None,{'PHONE_NUM': ['+1 212 555 0188']}
2,<s>[INST] I’m managing team of analysts suppor...,"{'max_tokens': 4000, 'n': 1, 'temperature': 1,...",As a manager of a team of analysts who suppor...,As a manager of a team of analysts who suppor...,"{'_type': 'html-file', 'path': 'media/html/d70...","[ , As, a, manager, of, a, team, of, analysts,...","[False, True, True, True, True, True, True, Tr...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",True,None,False,None,None,None,None,"{'NAME_STUDENT': ['James Brown'], 'STREET_ADDR..."
3,<s>[INST] Assignment: Mindmapping\n\nChallenge...,"{'max_tokens': 4000, 'n': 1, 'temperature': 1,...",Assignment: Mindmapping\n\nChallenge\n\nThe c...,Assignment: Mindmapping\n\nChallenge\n\nThe c...,"{'_type': 'html-file', 'path': 'media/html/2c0...","[ , Assignment, :, Mindmapping, \n\n, Challeng...","[False, False, True, False, False, False, Fals...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",None,None,False,False,None,None,None,"{'USERNAME': ['john42'], 'NAME_STUDENT': ['Jam..."
4,<s>[INST] Challenges\n\nI am leading an educat...,"{'max_tokens': 4000, 'n': 1, 'temperature': 1,...",As the leader of the educational domain entit...,As the leader of the educational domain entit...,"{'_type': 'html-file', 'path': 'media/html/d99...","[ , As, the, leader, of, the, educational, dom...","[False, True, True, True, True, True, True, Tr...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST...",None,None,False,False,False,None,None,"{'URL_PERSONAL': ['https://example.com'], 'USE..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5857,<s>[INST] Effective workflow management within...,"{'max_tokens': 4000, 'n': 1, 'temperature': 1,...",Challenge:\n\nAs a member of the online marke...,Challenge:\n\nAs a member of the online marke...,"{'_type': 'html-file', 'path': 'media/html/c64...","[ , Challenge, :, \n\n, As, a, member, of, the...","[False, False, False, False, True, True, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",None,None,False,None,None,False,None,"{'ID_NUM': ['123456789'], 'NAME_STUDENT': ['Ja..."
5858,<s>[INST] Visualization Tool\n\nChallenge\n\nM...,"{'max_tokens': 4000, 'n': 1, 'temperature': 1,...","As an art director, my latest project involve...","As an art director, my latest project involve...","{'_type': 'html-file', 'path': 'media/html/80e...","[ , As, an, art, director, ,, my, latest, proj...","[False, True, True, True, False, True, True, T...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",None,None,True,False,False,None,None,"{'URL_PERSONAL': ['https://example.com'], 'USE..."
5859,<s>[INST] Challenges\n\nWe are a business whic...,"{'max_tokens': 4000, 'n': 1, 'temperature': 1,...","As a Training Business Manager at my company,...","As a Training Business Manager at my company,...","{'_type': 'html-file', 'path': 'media

Select random fake data-rows

In [35]:
n_unique_ent_combs = len(generated_ents_combs_df)
n_ent_combs = len(generated_texts_df)

rand_comb_indexes = np.random.randint(n_unique_ent_combs, size=n_ent_combs)

generated_texts_df["label2ent"] = pd.Series(generated_ents_combs_df.to_numpy()[rand_comb_indexes])
generated_texts_df["label2ent"].head(2)

0    {'ID_NUM': ['685151882WLYFUvVuGGnonili'], 'NAM...
1    {'ID_NUM': ['902752678X6BuhIsQoRHDgRTr'], 'NAM...
Name: label2ent, dtype: object

Replace labels with ents

In [37]:
if 'label2position' in generated_texts_df.columns:
    generated_texts_df = generated_texts_df.agg(replace_labels_with_ents, axis=1)
else:
    generated_texts_df = generated_texts_df.agg(replace_ents_with_labels, axis=1) \
                                           .agg(tokenize_df_with_spacy, axis=1) \
                                           .agg(mark_ent_label_tokens, axis=1) \
                                           .agg(replace_labels_with_ents, axis=1)

generated_texts_df.head(2)

,prompt,sampling_params,generated_text,generated_text_with_ent_labels,vizualization,tokens,trailing_whitespace,labels,STREET_ADDRESS,PHONE_NUM,NAME_STUDENT,USERNAME,URL_PERSONAL,ID_NUM,EMAIL,true_ents_dict,label2ent,ents_present_in_generated_text,label2position
0,<s>[INST] Mind Mapping Tool\n\nChallenge:\n\nE...,"{'max_tokens': 4000, 'n': 1, 'temperature': 1,...","In the bustling city of London, at the famous...","In the bustling city of London, at the famous...","{'_type': 'html-file', 'path': 'media/html/361...","[ , In, the, bustling, city, of, London, ,, at...","[False, True, True, True, True, True, False, T...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O]",True,None,None,None,None,None,None,"{'STREET_ADDRESS': ['221B, Baker Street, Londo...","{'ID_NUM': ['685151882WLYFUvVuGGnonili'], 'NAM...",{'STREET_ADDRESS': False},"{'PHONE_NUM': [], 'ID_NUM': [], 'URL_PERSONAL'..."
1,<s>[INST] Project Title: Learning Modalities i...,"{'max_tokens': 4000, 'n': 1, 'temperature': 1,...",Project Title: Learning Modalities in the Pan...,Project Title: Learning Modalities in the Pan...,"{'_type': 'html-file', 'path': 'media/html/f9e...","[ , Project, Title, :, Learning, Modalities, i...","[False, True, False, True, True, True, True, T...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",None,True,None,None,None,None,None,{'PHONE_NUM': ['+1 212 555 0188']},"{'ID_NUM': ['902752678X6BuhIsQoRHDgRTr'], 'NAM...",{'PHONE_NUM': False},"{'PHONE_NUM': [], 'ID_NUM': [], 'URL_PERSONAL'..."


In [60]:
generated_texts_df["has_ents"] = generated_texts_df["labels"].apply(lambda labels: len(set(labels)) > 3)

In [61]:
generated_texts_df["has_ents"].value_counts()

has_ents
False    3452
True     2410
Name: count, dtype: int64

In [65]:
generated_texts_df = generated_texts_df[generated_texts_df["has_ents"]]

In [66]:
generated_texts_df['full_text'] = generated_texts_df['generated_text']

/tmp/ipykernel_3547505/903097803.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  generated_texts_df['full_text'] = generated_texts_df['generated_text']


In [67]:
generated_texts_df.to_json('../data/essay/rewrited_train.json')

In [52]:
row = generated_texts_df.iloc[130]
html = visualize_ents(row["tokens"], row["trailing_whitespace"], row["labels"])
display(HTML(html))

# Saving

In [ ]:
save_path = f"{texts_load_path.name}_" 
generated_texts_df.to_records(,index=False)